In [24]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

In [25]:
class FeatureMapDF(FeatureMap):
    def __init__(self):
        super().__init__()

    def get_df(self):
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f)

        def extractMetaData(f: Feature):
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    # TODO what else
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), f.getOverallQuality(), f.getIntensity()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getOverallQuality(), f.getIntensity()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('quality', 'f'), ('intensity', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        return pd.DataFrame(mdarr).set_index('id')

In [26]:
fmap = FeatureMapDF()
from urllib.request import urlretrieve
urlretrieve ("https://raw.githubusercontent.com/OpenMS/OpenMS/develop/share/OpenMS/examples/FRACTIONS/BSA1_F1.featureXML", "BSA1.featureXML")

FeatureXMLFile().load("BSA1.featureXML", fmap)
    

In [27]:
display(fmap.get_df())

,sequence,charge,RT,mz,quality,intensity
id,,,,,,
9650885788371886430,None,2,1942.600098,395.239288,0.808494,1.575720e+08
18416216708636999474,None,2,1749.138306,443.711212,0.893553,5.406930e+07
4391849314104099247,None,3,1850.058960,325.491180,0.788273,5.271700e+07
8826587690601962141,None,1,1977.238281,388.253906,0.844645,3.802820e+07
446423704489322421,None,2,1901.577637,738.311035,0.770521,3.498880e+07
...,...,...,...,...,...,...
10302832019150983352,None,3,1606.777588,426.142517,0.838608,3.678150e+04
8874676592351452214,None,3,1531.052490,403.509857,0.806600,3.128000e+04
2099235613780550198,None,1,1679.266113,344.091949,0.750061,3.097710e+04
